In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
train_df = pd.read_csv('training_dataset.csv')
study_df = pd.read_csv('study_dataset.csv')
val_df = pd.read_csv('validation_dataset.csv')

In [ ]:
labels = sorted(train_df['Ground_Truth'].unique())
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for i, label in enumerate(labels)}

In [ ]:
train_df['label'] = train_df['Ground_Truth'].map(label2id)
study_df['label'] = study_df['Ground_Truth'].map(label2id)
val_df['label'] = val_df['Ground_Truth'].map(label2id)

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
study_dataset = Dataset.from_pandas(study_df)
val_dataset = Dataset.from_pandas(val_df)

Tokenization

In [ ]:
model_name = 'distilroberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["Input_Text"], padding="max_length", truncation=True)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_study_dataset = study_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/16550 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/1839 [00:00<?, ? examples/s]

In [ ]:
class_weights = compute_class_weight('balanced', classes=np.unique(train_df['Ground_Truth']), y=train_df['Ground_Truth'])
weights = torch.tensor(class_weights, dtype=torch.float).to('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = torch.nn.CrossEntropyLoss(weight=weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

Model Training

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
)


model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    report = classification_report(labels, predictions, target_names=list(label2id.keys()), output_dict=True, zero_division=0)

    return {
        'accuracy': report['accuracy'],
        'macro_f1': report['macro avg']['f1-score'],
    }

In [ ]:
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-2888251520.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,1.489800,1.488333,0.902121,0.189708
2,1.406500,1.367315,0.414356,0.163803
3,1.362800,1.326238,0.489940,0.184676
4,1.326800,1.370045,0.584013,0.212955
5,1.304100,1.360871,0.592713,0.203043
6,1.308400,1.434468,0.779772,0.207593
7,1.245700,1.440857,0.740620,0.229240
8,1.220500,1.481422,0.780859,0.246084
9,1.183200,1.508476,0.815117,0.235439
10,1.157300,1.485634,0.778684,0.234659


TrainOutput(global_step=10350, training_loss=1.2954902007614357, metrics={'train_runtime': 7776.1022, 'train_samples_per_second': 21.283, 'train_steps_per_second': 1.331, 'total_flos': 2.192452739328e+16, 'train_loss': 1.2954902007614357, 'epoch': 10.0})

In [ ]:
predictions = trainer.predict(tokenized_study_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=-1)


In [ ]:
study_df['AI_Prediction_ID'] = predicted_labels
study_df['AI_Prediction'] = study_df['AI_Prediction_ID'].map(id2label)
study_df['Is_Correct'] = (study_df['Ground_Truth'] == study_df['AI_Prediction'])

In [ ]:
study_df.head()

,Date,Input_Text,Ground_Truth,label,AI_Prediction_ID,AI_Prediction,Is_Correct
0,2015-06-01,"Context: [Quarter: Q2, VIX: 13.97, Price vs 50...",Stable,4,4,Stable,True
1,2020-05-20,"Context: [Quarter: Q2, VIX: 27.99, Price vs 50...",Stable,4,4,Stable,True
2,2023-02-06,"Context: [Quarter: Q1, VIX: 19.43, Price vs 50...",Fall,2,4,Stable,False
3,2019-08-14,"Context: [Quarter: Q3, VIX: 22.10, Price vs 50...",Stable,4,4,Stable,True
4,2023-05-11,"Context: [Quarter: Q2, VIX: 16.93, Price vs 50...",Stable,4,4,Stable,True


In [ ]:
study_df.to_csv('study_dataset_with_predictions.csv', index=False)